In [1]:
import zipfile

zip_path = "/content/multi-class-garbage-classification.zip"
extract_path = "/content/multi-class-garbage-classification"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
# !pip install --upgrade torch

In [10]:
import os
import torch
from torchvision import transforms, datasets

data_dir = "/content/multi-class-garbage-classification/content/multi-class-garbage-classification"
IMG_SIZE = 224
BATCH_SIZE = 32

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset using ImageFolder
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

In [11]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

targets = [sample[1] for sample in dataset.samples]

train_idx, val_idx = train_test_split(
    range(len(dataset)), test_size=0.2, stratify=targets, random_state=42
)

train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [12]:
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load("/content/mobilenetv3_garbage_classifier.pt", weights_only=False)
model = model.to(device)

In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [14]:
from tqdm import tqdm

EPOCHS = 10

for epoch in range(EPOCHS):
    # --- Training ---
    model.train()
    train_loss, train_correct, train_total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = outputs.max(1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_acc = 100 * train_correct / train_total
    avg_train_loss = train_loss / len(train_loader)

    # --- Validation ---
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = outputs.max(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    avg_val_loss = val_loss / len(val_loader)

    # --- Print Epoch Summary ---
    print(
        f"Epoch {epoch+1}: "
        f"Train Loss = {avg_train_loss:.4f} | Train Acc = {train_acc:.2f}% || "
        f"Val Loss = {avg_val_loss:.4f} | Val Acc = {val_acc:.2f}%"
    )

Epoch 1/10 [Val]: 100%|██████████| 38/38 [00:04<00:00,  8.41it/s]


Epoch 1: Train Loss = 1.9617 | Train Acc = 59.79% || Val Loss = 0.4065 | Val Acc = 85.83%


Epoch 2/10 [Val]: 100%|██████████| 38/38 [00:03<00:00, 10.07it/s]


Epoch 2: Train Loss = 0.2639 | Train Acc = 91.33% || Val Loss = 0.2190 | Val Acc = 93.08%


Epoch 3/10 [Val]: 100%|██████████| 38/38 [00:03<00:00,  9.84it/s]


...

Epoch 8: Train Loss = 0.0161 | Train Acc = 99.60% || Val Loss = 0.1685 | Val Acc = 95.92%


Epoch 9/10 [Val]: 100%|██████████| 38/38 [00:03<00:00, 10.27it/s]


Epoch 9: Train Loss = 0.0106 | Train Acc = 99.71% || Val Loss = 0.1581 | Val Acc = 96.42%


Epoch 10/10 [Val]: 100%|██████████| 38/38 [00:04<00:00,  8.62it/s]

Epoch 10: Train Loss = 0.0085 | Train Acc = 99.77% || Val Loss = 0.1838 | Val Acc = 95.58%


In [17]:
from sklearn.metrics import classification_report

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate report
target_names = dataset.classes  # from ImageFolder — maps class indices to names
report = classification_report(all_labels, all_preds, target_names=target_names, digits=3)
print(report)

                 precision    recall  f1-score   support

          glass      0.944     0.935     0.940       200
          metal      0.945     0.945     0.945       200
  organic_waste      0.985     0.990     0.988       200
paper_cardboard      0.985     0.980     0.982       200
        plastic      0.957     0.900     0.928       200
          trash      0.921     0.985     0.952       200

       accuracy                          0.956      1200
      macro avg      0.956     0.956     0.956      1200
   weighted avg      0.956     0.956     0.956      1200



In [16]:
torch.save(model, "/content/mobilenetv3_garbage_classifier_2.pt")